In [1]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv('/home/santhosh/Projects/courses/Pinnacle/.env')

True

In [4]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# File Loader

In [5]:
import json

In [6]:
from llama_index.core import SimpleDirectoryReader

https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt

In [7]:
documents = SimpleDirectoryReader(input_files=["./data/paul_graham_essay.txt"], 
                                  filename_as_id=True).load_data(show_progress=True)

Loading files: 100%|██████████| 1/1 [00:00<00:00, 113.98file/s]


In [8]:
type(documents)

list

In [9]:
len(documents)

1

In [10]:
documents[0].to_dict().keys()

dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'text', 'mimetype', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator', 'class_name'])

In [11]:
documents[0].id_

'data/paul_graham_essay.txt'

In [12]:
documents[0].doc_id

'data/paul_graham_essay.txt'

In [13]:
documents[0].node_id

'data/paul_graham_essay.txt'

In [14]:
documents[0].hash

'17d7c59760a41d9abe07ffe931fd7ee08790e081aff1789642dc0351e954585f'

In [15]:
documents[0].metadata

{'file_path': 'data/paul_graham_essay.txt',
 'file_name': 'paul_graham_essay.txt',
 'file_type': 'text/plain',
 'file_size': 75042,
 'creation_date': '2024-04-16',
 'last_modified_date': '2024-04-15'}

In [16]:
documents[0].metadata.update({'author': 'paul_graham'})

In [17]:
documents[0].metadata

{'file_path': 'data/paul_graham_essay.txt',
 'file_name': 'paul_graham_essay.txt',
 'file_type': 'text/plain',
 'file_size': 75042,
 'creation_date': '2024-04-16',
 'last_modified_date': '2024-04-15',
 'author': 'paul_graham'}

In [18]:
documents[0].excluded_llm_metadata_keys

['file_name',
 'file_type',
 'file_size',
 'creation_date',
 'last_modified_date',
 'last_accessed_date']

In [19]:
documents[0].text

'\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then s

# Ingestion Pipeline

In [20]:
import tiktoken

In [21]:
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler

In [22]:
from llama_index.core import VectorStoreIndex, StorageContext

In [23]:
from llama_index.core.node_parser import SentenceSplitter, SentenceWindowNodeParser

In [24]:
from llama_index.core.ingestion import IngestionPipeline

In [25]:
from llama_index.embeddings.openai import OpenAIEmbedding

In [26]:
from llama_index.core.storage.docstore import SimpleDocumentStore

In [27]:
from llama_index.vector_stores.chroma import ChromaVectorStore

In [28]:
import chromadb

In [29]:
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode,
    verbose=True
)

In [30]:
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4o").encode,
    verbose=True
)

In [31]:
token_counter.reset_counts()

In [32]:
token_counter.total_embedding_token_count

0

## create

In [33]:
doc_store = SimpleDocumentStore()

In [34]:
doc_store.docs

{}

In [35]:
for i in doc_store.docs.keys():
    print(i)

In [36]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [37]:
chroma_client.count_collections()

0

In [38]:
chroma_collection = chroma_client.get_or_create_collection("paul_essay")

In [39]:
chroma_client.list_collections()

[Collection(name=paul_essay)]

In [40]:
chroma_collection.count()

0

In [41]:
chroma_collection.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [42]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [43]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [44]:
pipeline = IngestionPipeline(
    transformations = [SentenceSplitter(chunk_size=512, chunk_overlap=128),
                       OpenAIEmbedding(model_name='text-embedding-3-small', callback_manager=CallbackManager([token_counter]))],
    docstore=doc_store,
    vector_store=vector_store
)

In [45]:
pipeline.transformations[0].dict()

{'include_metadata': True,
 'include_prev_next_rel': True,
 'chunk_size': 512,
 'chunk_overlap': 128,
 'separator': ' ',
 'paragraph_separator': '\n\n\n',
 'secondary_chunking_regex': '[^,.;。？！]+[,.;。？！]?',
 'class_name': 'SentenceSplitter'}

In [46]:
nodes = pipeline.run(documents=documents, show_progress=True, num_workers=-1)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/47 [00:00<?, ?it/s]

Embedding Token Usage: 21949


In [47]:
doc_store.persist('./document storage/doc_store.json')

In [48]:
token_counter.total_embedding_token_count

21949

## Load

In [33]:
doc_store = SimpleDocumentStore.from_persist_path('./document storage/doc_store.json')

In [34]:
doc_store.docs

{'data/paul_graham_essay.txt': Document(id_='data/paul_graham_essay.txt', embedding=None, metadata={'file_path': 'data/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-16', 'last_modified_date': '2024-04-15', 'author': 'paul_graham'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first p

In [35]:
for i in doc_store.docs.keys():
    print(i)

data/paul_graham_essay.txt


In [36]:
doc_store.docs['data/paul_graham_essay.txt'].hash

'48f3b379e825cf288ae03eb7b6746d9f6703c698ab5b0ec7b83426fa8725d5dc'

In [37]:
documents[0].hash

'48f3b379e825cf288ae03eb7b6746d9f6703c698ab5b0ec7b83426fa8725d5dc'

In [38]:
doc_store.get_all_document_hashes()

{'48f3b379e825cf288ae03eb7b6746d9f6703c698ab5b0ec7b83426fa8725d5dc': 'data/paul_graham_essay.txt'}

In [39]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [40]:
chroma_client.count_collections()

1

In [41]:
chroma_collection = chroma_client.get_or_create_collection("paul_essay")

In [42]:
chroma_client.list_collections()

[Collection(name=paul_essay)]

In [43]:
chroma_collection.count()

47

In [44]:
chroma_collection.get()

{'ids': ['05ffd0b1-1bcc-4fc1-adf6-d00ca14ba8a3',
  '0e99333f-5d85-4eac-9bf3-8e1702e73e45',
  '0feabd85-759d-467c-9370-9145573e668a',
  '12124b7a-6b9f-4761-a36a-d20d374f5ec5',
  '19b26c9f-977d-4444-9857-998b5ae1986c',
  '21e3d7ce-75e1-4847-9197-2ab0b9210dad',
  '29bb4441-e404-431f-a474-0bf23a191063',
  '29cebd28-0234-432c-aeea-74cc32639cf8',
  '2aeb9044-3857-4fad-8541-b96b45bfb7fc',
  '309fbdb4-eb98-43de-9580-3890dc6d1128',
  '3193e1a8-a068-4d10-94a3-3432e6ceaacd',
  '32005ce9-b109-4fb8-9928-dedf64eee000',
  '3540f211-0fdc-429d-b366-0484188a6c4e',
  '42d97852-fdee-459c-b3af-e541bc4eef92',
  '47e36330-f7c7-41ba-8048-8782a9c14b5c',
  '5217fe28-a8ae-4ff9-bf73-4143e30f28a7',
  '560af57d-391d-4886-b434-68532964aa11',
  '57d4e51c-ed24-4af0-9c3c-71989852b953',
  '5a2dfe6d-c8a3-464d-99ab-5add5032eb2f',
  '5c5630f3-4acf-4e73-b722-e3bda9964a03',
  '5db817b4-e925-454f-a6af-1d323c7d7e6a',
  '5e784a9a-b642-4dde-8933-244ce96d2652',
  '6880f29f-8015-460a-ba53-fb3e27b3fd71',
  '6a5d88bd-65a3-4394-adba-

In [45]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [46]:
pipeline = IngestionPipeline(
    transformations = [SentenceSplitter(chunk_size=512, chunk_overlap=128),
                       OpenAIEmbedding(model_name='text-embedding-3-small', callback_manager=CallbackManager([token_counter]))],
    docstore=doc_store,
    vector_store=vector_store
)

In [47]:
pipeline.transformations[0].dict()

{'include_metadata': True,
 'include_prev_next_rel': True,
 'chunk_size': 512,
 'chunk_overlap': 128,
 'separator': ' ',
 'paragraph_separator': '\n\n\n',
 'secondary_chunking_regex': '[^,.;。？！]+[,.;。？！]?',
 'class_name': 'SentenceSplitter'}

In [48]:
nodes = pipeline.run(documents=documents, show_progress=True, num_workers=-1)

Parsing nodes: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

In [49]:
token_counter.total_embedding_token_count

0

In [50]:
chroma_client.list_collections()[0].dict()

{'name': 'paul_essay',
 'id': UUID('132aba60-3ced-499a-aee9-191764d7e30b'),
 'metadata': None,
 'tenant': 'default_tenant',
 'database': 'default_database'}

In [51]:
vector_store.to_dict()

{'stores_text': True,
 'is_embedding_query': True,
 'flat_metadata': True,
 'collection_name': None,
 'host': None,
 'port': None,
 'ssl': False,
 'headers': None,
 'persist_dir': None,
 'collection_kwargs': {},
 'class_name': 'ChromaVectorStore'}

## check

In [52]:
chroma_collection.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

In [53]:
chroma_collection.get()['metadatas'][0].keys()

dict_keys(['_node_content', '_node_type', 'author', 'creation_date', 'doc_id', 'document_id', 'file_name', 'file_path', 'file_size', 'file_type', 'last_modified_date', 'ref_doc_id'])

In [54]:
chroma_collection.get()

{'ids': ['05ffd0b1-1bcc-4fc1-adf6-d00ca14ba8a3',
  '0e99333f-5d85-4eac-9bf3-8e1702e73e45',
  '0feabd85-759d-467c-9370-9145573e668a',
  '12124b7a-6b9f-4761-a36a-d20d374f5ec5',
  '19b26c9f-977d-4444-9857-998b5ae1986c',
  '21e3d7ce-75e1-4847-9197-2ab0b9210dad',
  '29bb4441-e404-431f-a474-0bf23a191063',
  '29cebd28-0234-432c-aeea-74cc32639cf8',
  '2aeb9044-3857-4fad-8541-b96b45bfb7fc',
  '309fbdb4-eb98-43de-9580-3890dc6d1128',
  '3193e1a8-a068-4d10-94a3-3432e6ceaacd',
  '32005ce9-b109-4fb8-9928-dedf64eee000',
  '3540f211-0fdc-429d-b366-0484188a6c4e',
  '42d97852-fdee-459c-b3af-e541bc4eef92',
  '47e36330-f7c7-41ba-8048-8782a9c14b5c',
  '5217fe28-a8ae-4ff9-bf73-4143e30f28a7',
  '560af57d-391d-4886-b434-68532964aa11',
  '57d4e51c-ed24-4af0-9c3c-71989852b953',
  '5a2dfe6d-c8a3-464d-99ab-5add5032eb2f',
  '5c5630f3-4acf-4e73-b722-e3bda9964a03',
  '5db817b4-e925-454f-a6af-1d323c7d7e6a',
  '5e784a9a-b642-4dde-8933-244ce96d2652',
  '6880f29f-8015-460a-ba53-fb3e27b3fd71',
  '6a5d88bd-65a3-4394-adba-

In [55]:
chroma_collection.get()['metadatas'][0]['_node_content']

'{"id_": "05ffd0b1-1bcc-4fc1-adf6-d00ca14ba8a3", "embedding": null, "metadata": {"file_path": "data/paul_graham_essay.txt", "file_name": "paul_graham_essay.txt", "file_type": "text/plain", "file_size": 75042, "creation_date": "2024-04-16", "last_modified_date": "2024-04-15", "author": "paul_graham"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "data/paul_graham_essay.txt", "node_type": "4", "metadata": {"file_path": "data/paul_graham_essay.txt", "file_name": "paul_graham_essay.txt", "file_type": "text/plain", "file_size": 75042, "creation_date": "2024-04-16", "last_modified_date": "2024-04-15", "author": "paul_graham"}, "hash": "48f3b379e825cf288ae03eb7b6746d9f6703c698ab5b0ec7b83426fa8725d5dc", "class_name": "RelatedNodeInfo"}, "2": 

In [56]:
ids = chroma_collection.get()['ids']

In [57]:
for i in ids:
    data = json.loads(chroma_collection.get(i)['metadatas'][0]['_node_content'])
    print(data['relationships']['1']['node_id'])

data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
data/paul_graham_essay.txt
d

In [58]:
chroma_collection.get(ids=ids[0], include=['embeddings']).keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

In [59]:
chroma_collection.get(ids=ids[0],include=['embeddings', 'metadatas', 'documents'])

{'ids': ['05ffd0b1-1bcc-4fc1-adf6-d00ca14ba8a3'],
 'embeddings': [[0.007000122219324112,
   -0.00564673962071538,
   -0.013396192342042923,
   -0.014535480178892612,
   0.05743847042322159,
   -0.04547976702451706,
   -0.01905440166592598,
   0.05138265714049339,
   -0.04358350485563278,
   0.01619471237063408,
   0.06575756520032883,
   -0.028168706223368645,
   -0.005077095702290535,
   0.016255881637334824,
   -0.0023263650946319103,
   0.011171139776706696,
   0.011675790883600712,
   -0.03156363219022751,
   -0.009167827665805817,
   0.029376810416579247,
   0.021378854289650917,
   0.012042810209095478,
   -0.012341013178229332,
   0.036915991455316544,
   -0.008754931390285492,
   -0.029866168275475502,
   -0.0173263531178236,
   0.04636672884225845,
   0.020644817501306534,
   -0.03370457515120506,
   -0.01897794008255005,
   -0.016087664291262627,
   -0.021195346489548683,
   0.03844523802399635,
   -0.019803732633590698,
   0.022005846723914146,
   0.0200789961963892,
   -0.0

In [60]:
chroma_collection.get(ids=ids[0],include=['metadatas', 'documents'])['metadatas'][0].keys()

dict_keys(['_node_content', '_node_type', 'author', 'creation_date', 'doc_id', 'document_id', 'file_name', 'file_path', 'file_size', 'file_type', 'last_modified_date', 'ref_doc_id'])

In [61]:
chroma_collection.get(ids=ids[0],include=['metadatas', 'documents'])['metadatas'][0]

{'_node_content': '{"id_": "05ffd0b1-1bcc-4fc1-adf6-d00ca14ba8a3", "embedding": null, "metadata": {"file_path": "data/paul_graham_essay.txt", "file_name": "paul_graham_essay.txt", "file_type": "text/plain", "file_size": 75042, "creation_date": "2024-04-16", "last_modified_date": "2024-04-15", "author": "paul_graham"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "data/paul_graham_essay.txt", "node_type": "4", "metadata": {"file_path": "data/paul_graham_essay.txt", "file_name": "paul_graham_essay.txt", "file_type": "text/plain", "file_size": 75042, "creation_date": "2024-04-16", "last_modified_date": "2024-04-15", "author": "paul_graham"}, "hash": "48f3b379e825cf288ae03eb7b6746d9f6703c698ab5b0ec7b83426fa8725d5dc", "class_name": "Relate

In [62]:
chroma_collection.get(ids=ids, where={'file_size': {'$gt': 75040}}, 
                      where_document={'$contains': 'paul'}, include=['metadatas', 'documents'])

{'ids': ['29cebd28-0234-432c-aeea-74cc32639cf8',
  'cb011e7d-139a-4b0f-b573-7411de78dccd'],
 'embeddings': None,
 'metadatas': [{'_node_content': '{"id_": "29cebd28-0234-432c-aeea-74cc32639cf8", "embedding": null, "metadata": {"file_path": "data/paul_graham_essay.txt", "file_name": "paul_graham_essay.txt", "file_type": "text/plain", "file_size": 75042, "creation_date": "2024-04-16", "last_modified_date": "2024-04-15", "author": "paul_graham"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "data/paul_graham_essay.txt", "node_type": "4", "metadata": {"file_path": "data/paul_graham_essay.txt", "file_name": "paul_graham_essay.txt", "file_type": "text/plain", "file_size": 75042, "creation_date": "2024-04-16", "last_modified_date": "2024-04-

# Embeddings

In [63]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [65]:
embed_model = HuggingFaceEmbedding(model_name='BAAI/bge-base-en-v1.5')

In [66]:
embed_model.max_length

512

In [67]:
from llama_index.embeddings.openai import OpenAIEmbedding

In [68]:
embed_model = OpenAIEmbedding(model_name='text-embedding-3-small')

# LLM

In [69]:
from llama_index.llms.openai import OpenAI

In [71]:
gpt4om = OpenAI('gpt-4o-mini')

In [69]:
from llama_index.llms.huggingface import HuggingFaceInferenceAPI

In [39]:
llama3 = HuggingFaceInferenceAPI(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

In [42]:
mistral7b = HuggingFaceInferenceAPI(model_name="mistralai/Mistral-7B-Instruct-v0.2")

In [44]:
zephyr = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-beta")

# Index

In [159]:
from llama_index.llms.openai import OpenAI

In [160]:
from llama_index.core.retrievers import VectorIndexRetriever

In [161]:
from llama_index.core import get_response_synthesizer

In [162]:
from llama_index.core.response_synthesizers.type import ResponseMode

In [163]:
from llama_index.core.query_engine import RetrieverQueryEngine

In [164]:
from llama_index.core.chat_engine import ContextChatEngine, CondenseQuestionChatEngine, CondensePlusContextChatEngine

In [165]:
from llama_index.core.storage.chat_store import SimpleChatStore

In [166]:
from llama_index.core.memory import ChatMemoryBuffer

In [167]:
from llama_index.core import PromptTemplate

In [168]:
from llama_index.core.chat_engine.types import ChatMode

In [169]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

In [170]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [171]:
retriever = VectorIndexRetriever(index=index, similarity_top_k=5)

In [172]:
gpt_4o_mini = OpenAI(model = 'gpt-4o-mini')

In [173]:
qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

In [174]:
chat_text_qa_msgs = [
    ChatMessage(role=MessageRole.SYSTEM,
                content=("Only answer the question, if the question is answerable with the given context. \
                Otherwise say that question can't be answered using the context"),
                ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str)]

In [153]:
chat_text_qa_msgs = [
    ChatMessage(role=MessageRole.SYSTEM,
                content=("Only answer the question"),
                ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str)]

In [175]:
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [176]:
response_synthesizer = get_response_synthesizer(llm = gpt_35, response_mode=ResponseMode.COMPACT, 
                                                text_qa_template=text_qa_template, 
                                                callback_manager=CallbackManager([token_counter]))

In [177]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer)

In [184]:
query = "What is machine learning?"

In [185]:
query = "who is paul graham?"

In [141]:
Response = query_engine.query(query)

LLM Prompt Token Usage: 2437
LLM Completion Token Usage: 9


In [179]:
Response.response

"That question can't be answered using the context."

In [181]:
Response.response

'Paul Graham is an individual mentioned in the provided text, specifically as the author of the essay being referenced.'

In [182]:
token_counter.prompt_llm_token_count

34635

In [183]:
token_counter.completion_llm_token_count

258

In [138]:
for i, node in enumerate(Response.source_nodes):
    print(f"text of the node {i}")
    print(node.text)
    print(node.get_score())
    print("------------------------------------\n")

text of the node 0
Computer Science is an uneasy alliance between two halves, theory and systems. The theory people prove things, and the systems people build things. I wanted to build things. I had plenty of respect for theory — indeed, a sneaking suspicion that it was the more admirable of the two halves — but building things seemed so much more exciting.

The problem with systems work, though, was that it didn't last. Any program you wrote today, no matter how good, would be obsolete in a couple decades at best. People might mention your software in footnotes, but no one would actually use it. And indeed, it would seem very feeble work. Only people with a sense of the history of the field would even realize that, in its time, it had been good.

There were some surplus Xerox Dandelions floating around the computer lab at one point. Anyone who wanted one to play around with could have one. I was briefly tempted, but they were so slow by present standards; what was the point? No one el

In [125]:
Response.source_nodes[0].text

'Which in turn implies that people who are independent-minded (i.e. less influenced by custom) will have an advantage in fields affected by rapid change (where customs are more likely to be obsolete).\n\nHere\'s an interesting point, though: you can\'t always predict which fields will be affected by rapid change. Obviously software and venture capital will be, but who would have predicted that essay writing would be?\n\n[13] Y Combinator was not the original name. At first we were called Cambridge Seed. But we didn\'t want a regional name, in case someone copied us in Silicon Valley, so we renamed ourselves after one of the coolest tricks in the lambda calculus, the Y combinator.\n\nI picked orange as our color partly because it\'s the warmest, and partly because no VC used it. In 2005 all the VCs used staid colors like maroon, navy blue, and forest green, because they were trying to appeal to LPs, not founders. The YC logo itself is an inside joke: the Viaweb logo had been a white V o

## Chat

In [106]:
chat_store = SimpleChatStore()

In [107]:
chat_memory = ChatMemoryBuffer.from_defaults(token_limit=5000, chat_store=chat_store, llm=gpt_4o_mini)

In [109]:
ContextChatEngine(retriever=retriever, llm=gpt_4o_mini, memory=chat_memory)

TypeError: ContextChatEngine.__init__() missing 1 required positional argument: 'prefix_messages'

In [110]:
custom_prompt = PromptTemplate(
    """\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
"""
)

In [112]:
chat_engine = CondenseQuestionChatEngine(query_engine=query_engine, llm=gpt_4o_mini, 
                                         condense_question_prompt=custom_prompt, memory=chat_memory)

In [205]:
streaming_response = chat_engine.stream_chat("Who is Paul Graham?")

In [115]:
streaming_response = chat_engine.stream_chat("What is the context about")

LLM Prompt Token Usage: 78
LLM Completion Token Usage: 13
LLM Prompt Token Usage: 2447
LLM Completion Token Usage: 11


In [117]:
system_prompt = "Answer the question only based on the context provided"

In [141]:
chat_engine = CondensePlusContextChatEngine(retriever=retriever, llm=gpt_3_5, system_prompt=system_prompt, memory=chat_memory)

In [146]:
streaming_response = chat_engine.stream_chat("Who is Paul Graham?")

LLM Prompt Token Usage: 161
LLM Completion Token Usage: 5
LLM Prompt Token Usage: 1660
LLM Completion Token Usage: 85


In [145]:
streaming_response = chat_engine.stream_chat("What are his achievements?")

LLM Prompt Token Usage: 75
LLM Completion Token Usage: 6
LLM Prompt Token Usage: 1561
LLM Completion Token Usage: 79


In [142]:
streaming_response = chat_engine.stream_chat("what is machine learning?")

LLM Prompt Token Usage: 1558
LLM Completion Token Usage: 7


In [147]:
for token in streaming_response.response_gen:
    print(token, end="")

Paul Graham is a prominent figure in the tech and startup world. He is known for founding Viaweb, which was one of the first web-based applications, and later sold to Yahoo. Graham is also the co-founder of Y Combinator, a well-known startup accelerator that has supported numerous successful companies. He is recognized for his innovative approach to investing in startups and for his insightful essays on entrepreneurship and technology.

In [116]:
for token in streaming_response.response_gen:
    print(token, end="")

The question can't be answered using the context.

In [158]:
for token in streaming_response.response_gen:
    print(token, end="")

In [148]:
for i in chat_memory.chat_store.store['chat_history']:
    print(i.role.name)
    print(i.content)

USER
what is machine learning?
ASSISTANT
I don't know.
USER
What are his achievements?
ASSISTANT
Paul Graham's achievements include founding Viaweb, which was later acquired by Yahoo, and co-founding Y Combinator, a startup accelerator. He also established a unique approach to investing in startups, combining elements of angel investing and venture capital to provide comprehensive support to founders. Additionally, he pioneered the batch model of funding startups twice a year and offering intensive support over three months.
USER
Who is Paul Graham?
ASSISTANT
Paul Graham is a prominent figure in the tech and startup world. He is known for founding Viaweb, which was one of the first web-based applications, and later sold to Yahoo. Graham is also the co-founder of Y Combinator, a well-known startup accelerator that has supported numerous successful companies. He is recognized for his innovative approach to investing in startups and for his insightful essays on entrepreneurship and techno

In [151]:
Response = query_engine.query("Who is paul graham?")

AttributeError: 'TokenCountingHandler' object has no attribute 'event'

In [119]:
Response.response

'Paul Graham is an author and one of the founders of Y Combinator, an investment firm that focuses on funding startups.'

In [ ]:
chat_store.persist(persist_path="chat_store.json")
loaded_chat_store = SimpleChatStore.from_persist_path(
    persist_path="chat_store.json"
)

In [103]:
index = VectorStoreIndex(nodes=nodes, storage_context=storage_context, store_nodes_override=True)

In [52]:
index = VectorStoreIndex(nodes=nodes, store_nodes_override=True)

In [45]:
index.storage_context.persist(persist_dir="index")

In [96]:
nodes[0].ref_doc_id

'/home/santhosh/Projects/RAG/data/paul_graham_essay.txt'

In [56]:
nodes[0].to_dict().keys()

dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'text', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator', 'class_name'])

In [81]:
len(index.docstore.docs.values())

NameError: name 'index' is not defined

In [51]:
index.storage_context

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x77c550b63d50>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x77c546babfd0>, vector_stores={'default': <llama_index.core.vector_stores.simple.SimpleVectorStore object at 0x77c5451d5a50>, 'image': <llama_index.core.vector_stores.simple.SimpleVectorStore object at 0x77c545bd3f90>}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x77c5451ad650>)

In [47]:
nodes[-1].node_id

'2fc6008d-4ec4-4ca9-ba63-4d680b9285b1'

In [48]:
index.ref_doc_info

{'data/paul_graham_essay.txt': RefDocInfo(node_ids=['0f17135a-3916-41df-86c1-210f9d44eea0', 'f8bfd179-5245-49f6-8d6b-628b074ff44d', '596ccf32-90c1-4c1e-a971-ff96c96e410c', '0f28e4fa-848b-4120-a1f2-c73a8ac458ea', '27882781-9546-42b5-8b9a-a87ad78ef252', 'ec7a249f-333b-4463-829a-d0f9e4cb33b4', '29e670a3-8025-4ca2-8a91-7711cc17beb0', '12e3f0ca-19e7-424c-a2c9-3d2674a25cb7', 'f350365c-6f51-4ca0-ab43-9f6bdee4f684', 'e8bdf28b-204d-4866-babb-00f0be6736c2', 'c5e234f3-dd47-4d00-ba26-b5bd4116fe4b', '2b1e21b1-1f5e-4dfe-a40a-b3542ad900e1', 'cd5d0d18-ec60-4f81-86c4-bd119880884b', '6c72b7a2-5c08-4114-8344-2f43c7291a0b', 'fc64ddcf-ffd6-4e63-989f-2f3408b5a2c0', '9c062cc1-9dfc-446e-8dbc-5e0b914bb654', '616db433-f9b1-426a-b706-413035a1fe92', 'f45b55ea-0498-4dbb-8bc4-ec0fee1da31d', '69b878f6-743a-4553-be3d-6508e60319c8', '587ad45a-10bf-4d71-b0b0-36a84a85a92d', 'd2e24023-26be-43f3-a321-f50fbd9715b6', '71e323d5-0e63-4c92-8d2d-187ca96d4c90', 'e6a37990-e980-4508-aa97-74743bdb8dea', 'e3ced449-0bcb-4709-9e0e-9e8

In [66]:
index.docstore.docs.get('2e62035f-fa9a-43db-9609-629a6e106a2c').to_dict()

{'id_': '2e62035f-fa9a-43db-9609-629a6e106a2c',
 'embedding': None,
 'metadata': {'file_path': '/home/santhosh/Projects/RAG/data/paul_graham_essay.txt',
  'file_name': 'paul_graham_essay.txt',
  'file_type': 'text/plain',
  'file_size': 75042,
  'creation_date': '2024-04-16',
  'last_modified_date': '2024-04-15',
  'author': 'paul_graham'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: {'node_id': '/home/santhosh/Projects/RAG/data/paul_graham_essay.txt',
   'node_type': <ObjectType.DOCUMENT: '4'>,
   'metadata': {'file_path': '/home/santhosh/Projects/RAG/data/paul_graham_essay.txt',
    'file_name': 'paul_graham_essay.txt',
    'file_type': 'text/plain',
    'file_size': 75042,
    'creation_date':

In [57]:
chroma_collection.count()

47

In [36]:
index.as_chat_engine().chat('who is paul graham?').response

'Paul Graham is one of the founders of Y Combinator, an angel investment firm that focuses on funding and supporting startups.'

In [48]:
index.as_chat_engine().chat('who is paul graham?').response

"Paul Graham is an individual who co-founded Y Combinator, an investment firm that provides seed funding for startups. He played a key role in shaping Y Combinator's approach to investing in and supporting startups."

In [38]:
index.as_chat_engine().chat('who is paul graham?').sources[0].content

'Paul Graham is an individual who experimented with a new kind of still life painting technique, looked for an apartment to buy, and eventually came up with the idea of building a web app for making web apps. He founded a company named Aspra, which was later renamed to reflect the changing industry terminology. Additionally, he worked on developing a new dialect of Lisp called Arc and gave a talk at a Lisp conference about his experiences using Lisp at Viaweb.'

## Qdrant

In [91]:
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [92]:
import qdrant_client

In [93]:
client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333)

In [94]:
qdrant_vector_store = QdrantVectorStore(client=client, collection_name="paul_graham")

ResponseHandlingException: [Errno 111] Connection refused

In [48]:
from llama_index.vector_stores.milvus import MilvusVectorStore

In [49]:
vector_store = MilvusVectorStore(dim=1536, overwrite=True)

Failed to create new connection using: e9c83723eb5848cf9cea0e60bc23a9b8


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context


In [21]:
llm = OpenAI(model='gpt-3.5-turbo')

In [14]:
llm = Ollama(model='phi', request_timeout=180)

In [15]:
response = llm.complete("what is machine learning?")

In [16]:
response.text

' Machine learning is a type of artificial intelligence that allows computer systems to learn and improve from experience, without being explicitly programmed. It involves building algorithms that can analyze data and identify patterns, which are then used to make predictions or decisions. The goal is to create models that can adapt to new situations and continue to perform better over time.\n'

In [17]:
response = llm.complete("what are its applications?")

In [18]:
response.text

" As an AI language model, I don't have access to specific information about the name of the product you're referring to. However, in general, artificial intelligence has a wide range of applications across various industries such as healthcare, finance, education, transportation, and manufacturing. Some examples include:\n\n1. Healthcare: AI can be used for medical diagnosis, drug discovery, personalized medicine, and patient monitoring.\n2. Finance: AI can help with fraud detection, risk management, trading algorithms, and financial forecasting.\n3. Education: AI can assist in personalized learning, language translation, automated grading, and student assessment.\n4. Transportation: AI can help with autonomous vehicles, traffic optimization, and predictive maintenance for transportation systems.\n5. Manufacturing: AI can be used for quality control, process automation, predictive maintenance of machines, and supply chain management.\n\nThese are just a few examples, and the applicati

In [64]:
response = llm.stream_complete("what is machine learning?")

In [19]:
messages = [ChatMessage(role="system", content="You are a helpful assistant"),
            ChatMessage(role="user", content="What is machine learning?")]

In [20]:
response = llm.stream_chat(messages)

In [21]:
for r in response:
    print(r.delta, end="")

 Machine learning is a subfield of artificial intelligence (AI) that involves the development of algorithms and statistical models that enable computer systems to learn from data, identify patterns and make predictions or decisions without being explicitly programmed. It's used in many areas such as natural language processing, image recognition, speech recognition, and recommendation engines.


In [88]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [22]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import MilvusVectorStore

In [ ]:
vector_store = MilvusVectorStore()

In [90]:
index = VectorStoreIndex.from_documents(documents,service_context=service_context)

In [91]:
retriever = index.as_retriever()

In [97]:
retrieved_nodes = retriever.retrieve("What is the contribution of open-sourse to the infrastructure?")

In [98]:
len(retrieved_nodes)

2

In [99]:
print(retrieved_nodes[0].text)

65
ROADS AND BRIDGES: THE UNSEEN LABOR BEHIND OUR DIGITAL INFRASTRUCTUREOpEN SOURCE ’S CO mpLICATED RELATIONSHI p wITH m ONE y
influence or financial return.
Open source software is being created and used at a rate never seen 
before. Many open source projects are experiencing a difficult transi -
tion from selfless creative pursuit to critical public infrastructure. 
These increasing dependencies mean we have a shared responsibility 
to ensure that these projects find the support they need.


In [100]:
from llama_index.response_synthesizers import get_response_synthesizer

In [103]:
response_synthesizer = get_response_synthesizer(service_context=service_context)

In [104]:
query_engine = index.as_query_engine(retriever=retriever,
                                     response_synthesizer=response_synthesizer)

In [106]:
response = query_engine.query("What is the contribution of open-sourse to the infrastructure?")

ReadTimeout: timed out

In [ ]:
response.response

'The theme of DHS is "Infinite Possibilities: Exploring the Future with Generative AI".'

In [ ]:
response.source_nodes[1]

NodeWithScore(node=TextNode(id_='5d661538-29a1-4f80-ad89-1e08dd05b58a', embedding=None, metadata={'file_path': 'data/dhs.txt', 'file_name': 'dhs.txt', 'file_type': 'text/plain', 'file_size': 20789, 'creation_date': '2023-12-23', 'last_modified_date': '2023-12-23', 'last_accessed_date': '2023-12-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='70281eaa-6173-41b4-8517-0cd9233e295b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'data/dhs.txt', 'file_name': 'dhs.txt', 'file_type': 'text/plain', 'file_size': 20789, 'creation_date': '2023-12-23', 'last_modified_date': '2023-12-23', 'last_accessed_date': '2023-12-23'}, hash='8848374170c0535e3ed484f897e38f466a2dadf6565391abc4a7857369b55198'), <